# Entrega Final - Grupo 12
***

*Introducción a la Minería de Datos - Otoño 2018*

Pedro Belmonte,  
Jorge Fabry,  
Víctor Garrido,  
Pablo Ilabaca

__[GitHub](https://github.com/VictorPato/mineria-de-datos-entrega-3)__
***

# Introducción
***
## Motivación
La industria de los videojuegos es una de las más grandes industrias de entretenimiento a nivel mundial. Las mayores publicaciones se enfrentan codo a codo para conseguir el mayor éxito y con esto mayores ventas.

En esta industria, como en muchas otras, los críticos juegan un rol vital a la hora de definir la recepción que tendrá un juego. Casi siempre, los críticos reciben copias de juegos antes de que estos sean lanzados al público, por lo que tienen la primera palabra a la hora de publicitar si un juego es de calidad o no.

Dado esto, se da origen a un fenómeno en el que los críticos dan muy buena crítica a un juego, tal vez motivados por dinero o por quedar bien con los publicadores para seguir recibiendo acceso exclusivo a los juegos, y luego los usuarios dan un puntaje mucho menor, dejando una sensación de engaño y desencanto. A estos juegos con una gran diferencia de puntaje los llamaremos **fiascos**.

<img src="media/intro-ejemplos_de_fiascos.PNG" alt="Ejemplos de Fiascos" title="Puntaje de un par de fiascos en Metacritic" />

Interesa entonces utilizar las herramientas que provee este curso para estudiar los distintos patrones que pueden surgir a la hora de puntuar la calidad de un juego. En específico,  lograr crear un _predictor_ para saber, ojalá con bastante seguridad, si un juego será un **fiasco** o no.

## Hipótesis
Por la información manejada por los miembros del equipo, y lo que se ha observado de los últimos grandes fiascos en la industria de los videojuegos, se intuye que un factor importante a la hora de determinar si un juego será un fiasco o no será el publicador del juego. Esperamos que esta variable tenga alta correlación con los fiascos.

# Data Set
***
## Origen
Para explorar el fenómeno antes descrito, se utiliza el dataset extraido de https://www.kaggle.com/silver1996/videogames/data.

Este se construye de datos de Metacritic.com, el cual incluye 16719 entradas con los datos que se presentan a continuación.

In [ ]:
import pandas as pd
import numpy as np
original_data = pd.read_csv('data/Video_Games_Sales_as_at_22_Dec_2016.csv',encoding='latin1')
print("(Filas x Columnas) = ",original_data.shape)
original_data.head()

Los gráficos presentados a continuación se contruyen con este data set, con la intención de extraer información útil de este.

<img src="media/datos-publicadores_controversiales.jpg" alt="Publicadores controversiales" title="Publicadores con mayor diferencia de puntajes entre críticos y usuarios" />
<img src="media/datos-puntaje_por_genero.png" alt="Puntajes por género" title="Puntaje por género otorgado por los críticos de IGN" />


Se observa que el data set incluye muchas columnas con distintas variables, y también algunas filas que le faltan valores. Para comenzar a usar este data set se debe hacer una limpieza que solo deje datos que sean útiles.

## Limpieza
El siguiente script en R busca limpiar y ordenar el data set para ser utilizado posteriormente por los clasificadores. Se borran varias columnas que no estarían disponibles cuando sale un juego, como ventas. También se borran columnas que no nos aportarán información al explorar a futuro, como el año de lanzamiento, o el nombre del juego.

Se busca tambien limitar la cantidad de valores distintos de varias columnas, para mantener el problema con baja dimensionalidad.

Tras esto, se tienen 8 columnas. La primera un número por cada juego. Las siguientes 6 son parámetros, y la última la clase a clasificiar del juego. La clase corresponde a si el juego es un fiasco o no.

In [ ]:
data = pd.read_csv('data/data_para_clasificadores.csv',encoding='latin1')
data.head()

Cabe notar que las clases **no** están balanceadas.

In [ ]:
print("Cantidad de Fiascos")
data['Is_Fiasco'].value_counts()

## Adaptando el Data Set para clasificadores
Los clasificadores no pueden trabajar con Strings directamente. Vemos que Platform, Genre, Publisher y Rating son categorías que utilizan Strings, y hay que aplicar algún tipo de transformación para poder alimentarlas al clasificador.

Para esto, se utiliza un LabelBinarizer, que permite covertir las distintas categorías de una columna en columnas independientes. El resultado final es una matriz de dimensiones: 2348 rows x 34 columns.

In [ ]:
from sklearn import preprocessing

## Se aplica LabelBinarizer columna por columna, y finalmente se unen los resultados
## En header se van guardando los nombres de cada columna para luego agregarlas al nuevo Data Set
lb = preprocessing.LabelBinarizer()

lb.fit(data["Platform"])
platform = lb.transform(data["Platform"])
header = lb.classes_

lb.fit(data["Genre"])
genre = lb.transform(data["Genre"])
header = np.append(header,lb.classes_)

lb.fit(data["Publisher"])
publisher = lb.transform(data["Publisher"])
header = np.append(header,lb.classes_)

##sales = np.transpose(np.matrix(data["Global_Sales"].values))
##header = np.append(header,"Global_Sales")

critic_score = np.transpose(np.matrix(data["Critic_Score"].values))
header = np.append(header,"Critic_Score")

lb.fit(data["Rating"])
rating = lb.transform(data["Rating"])
header = np.append(header,lb.classes_)

fiasco = np.transpose(np.matrix(data["Is_Fiasco"].values))
header = np.append(header,"Is_Fiasco")

new_matrix = np.hstack((platform,genre,publisher,critic_score,rating,fiasco))
new_data = pd.DataFrame(new_matrix)
new_data.columns = header

## Se separa los datos de los resultados a predecir.
X = new_data[new_data.columns[:-1]]
y = new_data[new_data.columns[-1]]

Para observar la nueva data:

In [ ]:
new_data.head()

# Encontrando nuestro Predictor
***
## Experimentos básicos para elegir predictor
Como _predictor de fiascos_, se busca tener un clasificador que tenga un porcentaje alto de predicción de fiascos. Mediante varios experimentos, se muestra a continuación como se comparan varios clasificadores ante nuestro data set.

Utilizando código del laboratorio 2.2 del curso, se comparan distintos clasificadores mediante el contraste de las métricas promedio obtenidas tras un buen número de pruebas.

In [ ]:
import graphviz
import io
import pydotplus
import imageio

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split, cross_val_score
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, recall_score, precision_score
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import GaussianNB  # naive bayes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC  # support vector machine classifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [ ]:
def run_clf_with_cross_val(clf, X, y, num_tests=100, k=5):
    metrics = {'f1-score': [], 'precision': [], 'recall': [], 'score': []}
    
    for _ in range(num_tests):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, stratify=y)
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        scores = cross_val_score(clf, X, y, cv=k, scoring='f1')
        
        metrics['f1-score'].append(f1_score(y_test,predictions))
        metrics['recall'].append(recall_score(y_test,predictions))
        metrics['precision'].append(precision_score(y_test,predictions))
        metrics['score'].append(scores.mean())
    
    return metrics

In [ ]:
import warnings
warnings.filterwarnings('ignore')
def run_many_classifiers(X, y, num_test):
    c0 = ("Base Dummy", DummyClassifier(strategy='stratified'))
    c1 = ("Decision Tree", DecisionTreeClassifier(min_samples_split=100))
    c2 = ("Gaussian Naive Bayes", GaussianNB())
    c3 = ("KNN-3", KNeighborsClassifier(n_neighbors=3))
    c4 = ("KNN-5", KNeighborsClassifier(n_neighbors=5))
    c5 = ("Random Forest",RandomForestClassifier(max_features="auto", max_depth=15, n_estimators=40))


    classifiers = [c0, c1, c2, c3, c4, c5]
    print("Corriendo "+ str(num_test) + " tests por clasificador\n")

    for name, clf in classifiers:
        metrics = run_clf_with_cross_val(clf, X, y, num_test)
        print("----------------")
        print("Resultados para clasificador: ",name) 
        print("Precision promedio:",np.array(metrics['precision']).mean())
        print("Recall promedio:",np.array(metrics['recall']).mean())
        print("F1-score promedio:",np.array(metrics['f1-score']).mean())
        print("Cross Validation F1-score promedio:", np.array(metrics['score']).mean())
        
run_many_classifiers(X, y, 20)
warnings.filterwarnings('once')

Se puede observar que los resultados obtenidos no son considerablemente buenos. Tomando en cuenta el F1-score, que describe en general la eficacia de un clasificador, los puntajes son bien bajos, aunque aún así mejores que el base dummy. 

De todos los clasificadores explorados, Gaussan Naive Bayes y Random Forest son los que se ven más prometedores a predictor de fiascos.


## Utilizando Subsampling y Oversampling
En un intento de encontrar mejores resultados que los anteriores, se aplicarán estas estrategias sobre el dataset, buscando que los clasificadores aprendan mejor teniendo clases balanceadas.
Nuevamente nos apoyamos en código trabajado en el laboratorio 2.2.

In [ ]:
# oversampling sobre la clase True
idx = np.random.choice(new_data.loc[data.Is_Fiasco == True].index, size=1990)
data_oversampled = pd.concat([new_data, new_data.iloc[idx]])

print("Data oversampled on class 'True'")
print(data_oversampled['Is_Fiasco'].value_counts())
print()

# subsampling sobre la clase False
idx = np.random.choice(new_data.loc[new_data.Is_Fiasco == False].index, size=1990, replace=False)
data_subsampled = new_data.drop(new_data.iloc[idx].index)

print("Data subsampled on class 'False'")
print(data_subsampled['Is_Fiasco'].value_counts())

In [ ]:
warnings.filterwarnings('ignore')
# datos "oversampleados" 
X_over = data_oversampled[new_data.columns[:-1]]
y_over = data_oversampled[new_data.columns[-1]]

# datos "subsampleados"
X_subs = data_subsampled[new_data.columns[:-1]]
y_subs = data_subsampled[new_data.columns[-1]]

print("----------Prueba Oversampling------------")
run_many_classifiers(X_over, y_over, 20)

print("\n\n----------Prueba Subsampling------------")
run_many_classifiers(X_subs, y_subs, 20)

También, como experimento, se buscó hacer subsampling y oversampling al mismo tiempo para no repetir tantos datos, pero tampoco quedarnos con tan pocos. Esto se muestra a continuación.

In [ ]:
idx = np.random.choice(new_data.loc[data.Is_Fiasco == True].index, size=71)
data_master = pd.concat([new_data, new_data.iloc[idx]])
idx = np.random.choice(new_data.loc[new_data.Is_Fiasco == False].index, size=1669, replace=False)
data_master = data_master.drop(new_data.iloc[idx].index)
print("Data subsampled on class 'False' and oversampled on class 'True'")
print(data_master['Is_Fiasco'].value_counts())
X_mast = data_master[new_data.columns[:-1]]
y_mast = data_master[new_data.columns[-1]]
run_many_classifiers(X_mast, y_mast, 10)

## Gráficos de resultados obtenidos
A continuación se grafican los resultados obtenidos para Base Dummy, Decision Tree y Random Forest. Como se han corrido varias veces los clasificadores puede que los gráficos no correspondan perfectamente a los valores, pero sí con la cercanía suficiente para ser precisos.
<img src="media/clasificadores-datos_crudos.png" alt="" title="" />
<img src="media/clasificadores-datos_oversampling.png" alt="a" title="a" />
<img src="media/clasificadores-datos_subsampling.png" alt="" title="" />
<img src="media/clasificadores-datos_over_y_sub.png" alt="" title="" />

## Conclusión Clasificadores
Haciendo un análisis de los resultados obtenidos, podemos considerar que **Random Forest** es clasificador que tiene mejor desempeño en cuanto a resultados, en especial al hacer **oversampling**.

Si bien no es siempre certero, tiene un puntaje suficiente de exactitud, lo que consideramos un logro aceptable con respecto a lo que esperabamos obtener.
Si necesitaramos crear un predictor efectivo, utilizariamos ese clasificador.

## Graficando Decision Trees
En un ejercicio para explorar la importancia de las variables, y para observar la lógica de los decision trees, se grafican los árboles de decisión, donde los colores indican afinidad con una clase.

In [ ]:
c12= DecisionTreeClassifier(min_samples_split=100)
c13= DecisionTreeClassifier(min_samples_split=100)
c14= DecisionTreeClassifier(min_samples_split=100)
c15= DecisionTreeClassifier(min_samples_split=100)
features=new_data.columns[:-1]
train, test= train_test_split(new_data,test_size=.30, stratify=y)

        
X_train= train[features]
y_train=train["Is_Fiasco"]

X_test=test[features]
y_test=test["Is_Fiasco"]

dt12=c12.fit(X_train,y_train)
dt13=c13.fit(X_over,y_over)
dt14=c14.fit(X_subs,y_subs)
dt15=c15.fit(X_mast,y_mast)

def show_tree(tree, features, path):
    path = "images/" + path
    f= io.StringIO()
    export_graphviz(tree, out_file=f, feature_names=features,filled=True,rounded=True)
    pydotplus.graph_from_dot_data(f.getvalue()).write_png(path)
    img= imageio.imread(path)
    plt.rcParams["figure.figsize"]=(20,20)
    plt.imshow(img)

Árbol Normal:

In [ ]:
show_tree(dt12, features, 'arbol_normal.png')

Árbol Oversampled:

In [ ]:
show_tree(dt13, features, 'arbol_oversampled.png')

Árbol Subsampled:

In [ ]:
show_tree(dt14, features, 'arbol_subsampled.png')

Árbol con Oversampling y Subsampling:

In [ ]:
show_tree(dt15, features, 'arbol_master.png')

## Importancia de Atributos
Aplicando Random Forest, se pueden obtener los atributos que más seguido se utilizan para discriminar la clase a clasificar. De esta forma se puede evidenciar cuales son las columnas más importantes para decidir si un juego es un fiasco o no.

In [ ]:
import operator
clf6 = RandomForestClassifier(n_estimators= 1000,max_depth=100, random_state=0)
train, test= train_test_split(new_data,test_size=.30, stratify=y)
        
X_train= train[features]
y_train=train["Is_Fiasco"]

X_test=test[features]
y_test=test["Is_Fiasco"]
clf6.fit(X_train, y_train)
mi_lista_de_tuplas = []
for i in range(33):
    tupla = (header[i],clf6.feature_importances_[i])
    mi_lista_de_tuplas.append(tupla)
mi_lista_de_tuplas.sort(key=operator.itemgetter(1), reverse=True)
for i in range(33):
    print(mi_lista_de_tuplas[i])

Vemos con esto que la variable más importante por lejos es Critic_Score, lo cual tiene mucho sentido pues con esta en parte se define la clase Is_Fiasco. Luego, dentro de las más importantes tenemos una plataforma (PC), dos publicadores (Activision y EA) y dos géneros (Action y Sports).

# Clustering
***
Se intentó hacer clustering usando K-means con los atributos “Global_Sales”, “Critic_Score” y “User_Score”, usando el siguiente script de R:

Primero se consideran las ventas globales y la puntuación de la crítica. Usando el siguiente gráfico y observando los codos se decidió usar K = 3:
<img src="media/criticscoreglobalsales1.png" alt="" title="" />

Con K = 3 el resultado es el siguiente:

<img src="media/criticscoreglobalsales2.png" alt="" title="" />

Luego se consideran las ventas globales y puntuación de usuarios. Usando el siguiente gráfico también se decide usar K = 3

<img src="media/userscoreglobalsales1.png" alt="" title="" />

Con K = 3 el resultado es el siguiente:

<img src="media/userscoreglobalsales2.png" alt="" title="" />

Luego se consideran las puntuaciones de usuarios y críticos. Usando el siguiente gráfico también se decide usar K = 3

<img src="media/usercriticscore1.png" alt="" title="" />

Con K = 3 el resultado es el siguiente:

<img src="media/usercriticscore2.png" alt="" title="" />

Finalmente se hace un cluster tridimensional:

<img src="media/cluster3d.png" alt="" title="" />

# Reglas de Asociación
***
Se aplicaron técnicas de reglas de asociación. Para esto primero se hace un preprocesamiento de los datos, en donde, por ejemplo, se definen las clases respecto a la cantidad de ventas (Muchisimas, Muchas, Bastante, Intermedio, Pocas, Muypocas).

Después de generar estos archivos .csv se aplican las reglas de asociación:

Con esto se obtienen los siguientes resultados:
> Usando todos los datos se obtienen resultados que no aportan mucha información, como por ejemplo:
- {Muchas ventas Regionales} => {Muchas ventas globales}
- {Konami,  muy pocas ventas en EU} => {Konami Digital Entertainment}

> Considerando solo las ventas globales y no las regionales suceden cosas parecidas:
- {muy pocas Ventas globales,  Ubisoft Montreal} => {Ubisoft}

> Sin considerar las ventas tampoco se obtienen resultados muy significativos:
- {EA Canada, Puntajes usuarios alto} => {Electronic Arts}

> Sin el género, la plataforma ni el año:
- {Konami,  n° críticas usuarios muy bajo} => {Konami Digital Entertainment}

> Sin el número de críticas se obtienen resultados un poco más interesantes, como por ejemplo:
- {Nintendo,  Puntajes Críticos muy alto} => {Puntajes usuarios muy alto}

> Lo cual a primera vista podría indicar que los juegos de Nintendo tienden a no ser un fiasco, sin embargo, esto es un error pues ambos puntajes podrían ser muy altos según los criterios definidos y aún así tener una diferencia de 2 puntos.

> Al eliminar el desarrollador sucede algo parecido al caso anterior, aunque en el caso de Electronic Arts se observa que puntaje de usuarios alto suele estar relacionado con puntaje de críticos aún más alto:
- {Electronic Arts, Puntajes usuarios alto} => {Puntajes Críticos muy alto}
- {Nintendo, Puntajes Críticos muy alto} => {Puntajes usuarios muy alto}

> Con el desarrollador pero sin publicador no se observan cambios significativos.

> Al considerar el atributo Is_Fiasco se obtienen los resultados más interesantes, en los cuales se sugiere que el publicador puede estar muy relacionado con la clasificación del videojuego:
- {TRUE} => {Activision}
- {TRUE} => {Electronic Arts}
- {Namco Bandai Games} => {FALSE}
- {THQ} => {FALSE}

Esto último ayuda a validar las hipótesis iniciales.

# Conclusión
Si bien, se considera que el problema es bastante complejo, pues existen muchos factores difíciles de medir que influyen en lo que sucede en el lanzamiento de un videojuego, los resultados de los experimentos en general fueron interesantes y, en cierta medida, satisfactorios. 